In [2]:
ENV["PYTHONPATH"]="/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";

In [3]:
using Crystalline, MPBUtils, SymmetryBases, JLD2, PyPlot, HDF5

In [33]:
# Check permittivity and permeability 
sgs = [2, 6, 9:17...]
for sg in sgs
    println("Analyzing plane group: $sg"); flush(stdout)
    for mode in ["te", "tm"]
        for idx in 1:10
            dir = "./"
            original_dir = "../symeigs/output/sg$sg/eps3/$mode/"
            epsilon = h5open(dir*"dim2-sg$sg-$(idx)-res64-$mode-epsilon.h5", "r") do file
                read(file, "data")
            end
            mu = h5open(dir*"dim2-sg$sg-$idx-res64-$mode-mu.h5", "r") do file
                read(file, "data")
            end;
            epsilon_original = h5open(original_dir*"dim2-sg$sg-$(idx+20000)-res64-$mode-epsilon.h5", "r") do file
                read(file, "data")
            end
            @assert isapprox(epsilon, ones(64, 64))
            @assert isapprox(mu, epsilon_original)

        end
    end
end

Analyzing plane group: 2
Analyzing plane group: 6
Analyzing plane group: 9
Analyzing plane group: 10
Analyzing plane group: 11
Analyzing plane group: 12
Analyzing plane group: 13
Analyzing plane group: 14
Analyzing plane group: 15
Analyzing plane group: 16
Analyzing plane group: 17


In [29]:
sg = 10
idx = 10
mode = "te"
dir = "./"
original_dir = "../symeigs/output/sg$sg/eps3/$mode/"

mu = h5open(dir*"dim2-sg$sg-$idx-res64-$mode-mu.h5", "r") do file
                read(file, "data")
            end
epsilon_original = h5open(original_dir*"dim2-sg$sg-$(idx+20000)-res64-$mode-epsilon.h5", "r") do file
                read(file, "data")
end;

In [124]:
function em_duality_transform(n::Vector{<:Integer}, sg::Integer)
    n_prime = if sg in [2, 10, 13, 16]
        n
    elseif sg == 6
        [n[2], n[1], n[4], n[3], n[6], n[5], n[8], n[7], n[10], n[9], n[12], n[11], n[14], n[13], n[16], n[15], n[17]]
    elseif sg == 9
        [n[2], n[1], n[4], n[3], n[5], n[6], n[8], n[7], n[10], n[9], n[11]]
    elseif sg == 11
        [n[2], n[1], n[4], n[3], n[8], n[7], n[6], n[5], n[9], n[13], n[12], n[11], n[10], n[14], n[15]]
    elseif sg == 12
        [n[2], n[1], n[3], n[4], n[8], n[7], n[6], n[5], n[9], n[10]]
    elseif sg == 14
        [n[2], n[1], n[4], n[3], n[5], n[6], n[7], n[8], n[9]]
    elseif sg == 15
        [n[2], n[1], n[3], n[5], n[4], n[7], n[6], n[8], n[10], n[9], n[11], n[12]]
    elseif sg == 17
        [n[2], n[1], n[4], n[3], n[6], n[5], n[8], n[7], n[9], n[10] , n[12], n[11], n[13], n[14]]
    end
    return n_prime
end;

In [127]:
# Below we check that we should get the exact same topologies upon the em_duality transformation


In [126]:
for sg in [6, 9, 11, 12, 14, 15, 17]
    ns, brs = compatibility_basis(sg, 2)
    for n in ns
        n_prime = em_duality_transform(n, sg)
        @assert n_prime in ns
        original_topo = calc_detailed_topology(n, brs)
        transformed_topo = calc_detailed_topology(n, brs)
        @assert original_topo == transformed_topo
    end
end

In [128]:
# Explicitly check for 10 samples across TE/TM modes and across 11 plane groups

In [182]:
sgs = [2, 6, 9:17...]
for sg in sgs
    println("Analyzing plane group: $sg"); flush(stdout)
    for mode in ["te", "tm"]
        compare_mode = (mode == "te" ? "tm" : "te") # TE mapped to TM and vice versa
        dir = "./output/sg$sg/eps3/$mode/"
        compare_dir = "../symeigs/output/sg$sg/eps3/$(compare_mode)/"
        loaded_data = load(dir*"sg$sg-epsid3-$mode.jld2")
        loaded_data_compare = load(compare_dir*"sg$sg-epsid3-res64-$(compare_mode).jld2")
        summariesv = loaded_data["summariesv"]
        cumsummariesv = loaded_data["cumsummariesv"]
        summariesv_compare = loaded_data_compare["summariesv"]
        num_successes = 0
        for (phc_idx, (cv, sv, s_cv)) in enumerate(zip(cumsummariesv, summariesv, summariesv_compare))
            for (mult_idx, (c, s, s_c)) in enumerate(zip(cv, sv, s_cv))
                c.band[end] > 10 && continue
                n = s.n
                n_compare = s_c.n
                (n[end] != n_compare[end]) && break
                @assert n == em_duality_transform(n_compare, sg) "Failed EM duality for plane group $sg, photonic crystal index: $phc_idx and multiplet $mult_idx"
                num_successes += 1
            end
        end
        println("plane group $sg with mode $mode and num successes $num_successes"); flush(stdout)
    end
end

Analyzing plane group: 2
plane group 2 with mode te and num successes 97
plane group 2 with mode tm and num successes 98
Analyzing plane group: 6
plane group 6 with mode te and num successes 15
plane group 6 with mode tm and num successes 6
Analyzing plane group: 9
plane group 9 with mode te and num successes 46
plane group 9 with mode tm and num successes 18
Analyzing plane group: 10
plane group 10 with mode te and num successes 62
plane group 10 with mode tm and num successes 55
Analyzing plane group: 11
plane group 11 with mode te and num successes 26
plane group 11 with mode tm and num successes 29
Analyzing plane group: 12
plane group 12 with mode te and num successes 15
plane group 12 with mode tm and num successes 17
Analyzing plane group: 13
plane group 13 with mode te and num successes 66
plane group 13 with mode tm and num successes 65
Analyzing plane group: 14
plane group 14 with mode te and num successes 27
plane group 14 with mode tm and num successes 22
Analyzing plane gr